# Metrics

Implement metrics to evaluate clustering performance based on BCubed.

$$
P_b = \frac{1}{N} \sum_{i=1}^{N} \frac{1}{|C(i)|} \sum_{j \in C(i)}B(i, j)
\\
R_b = \frac{1}{N} \sum_{i=1}^{N} \frac{1}{|L(i)|} \sum_{j \in 
L(i)}B(i, j)
$$

Where $C(i)$ is the trackster that contains layercluster $i$, and $L(i)$ are all layerclusters of the same particle as layercluster $i$.

Then the F-sore is computed as:

$$
F_p = \frac{2P_bR_b}{P_b + R_b}
$$